# Poe x Frost 
* This project aims to conceive a classification strategy of differentiating Edgar Allan Poe's poems from Robert Frost's. We'll conduct the experiment by training two separate Markov Models, each of them calibrated with the poems of one of the poets.
* Then, given the $p(\text{text}|\text{author})$ returned by the models, we'll apply the Bayes' Theorem to compute $p(\text{author}|\text{text})$ to receive the final prediction.

## Loading the Documents

In [1]:
from re import sub
from typing import List

def load_text(filename:str)->List[str]:
    '''
        Reads the .txt file.
        
        Parameter
        ---------
        `filename`: str
            The name of the poems file.
            
        Returns
        -------
        A list containing each strophe's content.
    '''
    with open(f'/kaggle/input/poe-vs-frost/{filename}', 'r') as f:
        strophe_delim = '\n\n'
        return sub('\n\u2009\n', strophe_delim, f.read()).split(strophe_delim)
    
a = load_text('05_robert_frost.txt')
a[3]

'I shall be telling this with a sigh\nSomewhere ages and ages hence:\nTwo roads diverged in a wood, and I,\nI took the one less traveled by,\nAnd that has made all the difference.'

In [2]:
print(a[0])

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 
